In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from scipy.optimize import minimize
from fredapi import Fred

In [2]:
tickers = ["SPY", "BND", "GLD", "QQQ", "VTI"]

In [3]:
end_date = datetime.today()
print(end_date)

2024-07-13 11:39:37.900898


In [4]:
start_date = end_date - pd.to_timedelta(5 * 365, 'D')
print(start_date)

2019-07-15 11:39:37.900898


In [5]:
adj_close_df = pd.DataFrame()

In [6]:
for ticker in tickers:
    data = yf.download(ticker, start = start_date, end = end_date)
    adj_close_df[ticker] = data["Adj Close"]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

/Users/tina/anaconda3/envs/env_py_31011/lib/python3.10/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/tina/anaconda3/envs/env_py_31011/lib/python3.10/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

/Users/tina/anaconda3/envs/env_py_31011/lib/python3.10/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/tina/anaconda3/envs/env_py_31011/lib/python3.10/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



[*********************100%%**********************]  1 of 1 completed


/Users/tina/anaconda3/envs/env_py_31011/lib/python3.10/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [7]:
print(adj_close_df)

                   SPY        BND         GLD         QQQ         VTI
Date                                                                 
2019-07-15  277.972565  72.419434  133.529999  188.072952  141.750198
2019-07-16  277.076050  72.323067  132.399994  187.104248  141.390030
2019-07-17  275.190643  72.542107  134.580002  186.183960  140.457474
2019-07-18  276.198029  72.673546  136.479996  186.397110  140.974579
2019-07-19  274.663788  72.594704  134.470001  185.031219  140.171188
...                ...        ...         ...         ...         ...
2024-07-08  555.280029  72.400002  218.190002  497.339996  272.489990
2024-07-09  555.820007  72.370003  218.559998  497.769989  272.420013
2024-07-10  561.320007  72.430000  219.360001  502.959991  275.059998
2024-07-11  556.479980  72.790001  223.250000  491.929993  273.829987
2024-07-12  559.989990  72.980003  223.110001  494.820007  275.640015

[1258 rows x 5 columns]


In [8]:
log_returns = np.log(adj_close_df - adj_close_df.shift(1))

/Users/tina/anaconda3/envs/env_py_31011/lib/python3.10/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/Users/tina/anaconda3/envs/env_py_31011/lib/python3.10/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


In [9]:
log_returns.replace([np.inf, -np.inf], np.nan, inplace=True)
log_returns.isna().sum()

SPY    570
BND    616
GLD    585
QQQ    557
VTI    568
dtype: int64

In [10]:
log_returns.ffill()

,SPY,BND,GLD,QQQ,VTI
Date,,,,,
2019-07-15,NaN,NaN,NaN,NaN,NaN
2019-07-16,NaN,NaN,NaN,NaN,NaN
2019-07-17,NaN,-1.518501,0.779329,NaN,NaN
2019-07-18,0.007358,-2.029211,0.641851,-1.545759,-0.659509
2019-07-19,0.007358,-2.029211,0.641851,-1.545759,-0.659509
...,...,...,...,...,...
2024-07-08,-0.446264,-1.108657,1.078405,0.165508,-0.994265
2024-07-09,-0.616227,-1.108657,-0.994265,-0.843987,-0.994265
2024-07-10,1.704748,-2.813451,-0.223140,1.646734,0.970773


In [11]:
log_returns.dropna(inplace=True)
log_returns

,SPY,BND,GLD,QQQ,VTI
Date,,,,,
2019-07-18,0.007358,-2.029211,0.641851,-1.545759,-0.659509
2019-07-24,0.264840,-2.657304,-0.385673,0.275682,-0.141586
2019-08-06,1.302538,-1.690552,0.113338,0.896404,0.556883
2019-08-08,1.652907,-2.789081,-0.342502,1.356988,0.949935
2019-09-04,1.115168,-1.960411,-0.094307,0.954051,0.377699
...,...,...,...,...,...
2024-06-18,0.326104,-1.389351,-0.150822,-1.898790,-0.389180
2024-06-27,-0.150840,-2.043357,0.879628,0.215104,-0.434320
2024-07-03,0.896093,-0.891608,0.887895,1.401182,0.104347


In [12]:
log_returns

,SPY,BND,GLD,QQQ,VTI
Date,,,,,
2019-07-18,0.007358,-2.029211,0.641851,-1.545759,-0.659509
2019-07-24,0.264840,-2.657304,-0.385673,0.275682,-0.141586
2019-08-06,1.302538,-1.690552,0.113338,0.896404,0.556883
2019-08-08,1.652907,-2.789081,-0.342502,1.356988,0.949935
2019-09-04,1.115168,-1.960411,-0.094307,0.954051,0.377699
...,...,...,...,...,...
2024-06-18,0.326104,-1.389351,-0.150822,-1.898790,-0.389180
2024-06-27,-0.150840,-2.043357,0.879628,0.215104,-0.434320
2024-07-03,0.896093,-0.891608,0.887895,1.401182,0.104347


In [13]:
# Covariance matrix
cov_matrix = log_returns.cov() * 252
print(cov_matrix)

            SPY         BND         GLD         QQQ         VTI
SPY  274.087462   71.899198   67.601934  232.547554  266.716465
BND   71.899198  260.164275   93.500600   68.361604   62.866779
GLD   67.601934   93.500600  276.795267   65.840791   58.194692
QQQ  232.547554   68.361604   65.840791  346.682578  230.724652
VTI  266.716465   62.866779   58.194692  230.724652  291.410019


In [14]:
def standard_deviation(weights, cov_matrix):
    variance = weights.T @ cov_matrix @ weights
    return np.sqrt(variance)

In [15]:
def expected_return(weights, log_returns):
    return np.sum(log_returns.mean() * weights) * 252

$$
SharpeRatio = {portfolioReturn - riskFreeRate \over portfolioStd}
$$

In [16]:
def sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return (expected_return(weights, log_returns) - risk_free_rate) / standard_deviation(weights, cov_matrix)

In [50]:
# Define the ticker symbol for the 10-year Treasury bond yield
ticker = "^TNX"

# Fetch the data
ten_year_treasury = yf.Ticker(ticker)

# Get the historical market data (we'll just fetch the latest data point)
#hist = ten_year_treasury.history(start=end_date, end=end_date)
hist = ten_year_treasury.history(period="300d")

# Extract the latest closing price (which represents the yield)
risk_free_rate = hist['Close'].iloc[-1] / 100
risk_free_rate

/Users/tina/anaconda3/envs/env_py_31011/lib/python3.10/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


0.04189000129699707

In [56]:
def neg_sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return -sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate)

In [57]:
constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
bounds = [(0, 0.5) for _ in range(len(tickers))]

In [58]:
initial_weights = np.repeat(1/len(tickers), len(tickers))
print(initial_weights)

[0.2 0.2 0.2 0.2 0.2]


In [59]:
optimize_results = minimize(neg_sharpe_ratio, initial_weights, args = (log_returns, cov_matrix, risk_free_rate), method = 'SLSQP',
                            constraints=constraints, bounds=bounds)

In [60]:
optimal_weights = optimize_results.x

In [62]:
print("Optimal weights:")
for ticker, weight in zip(tickers, optimal_weights):
    print(f"{ticker}: {weight:.4f}")

print()

optimal_portfolio_return = expected_return(optimal_weights, log_returns)
optimal_portfolio_volatility = standard_deviation(optimal_weights, cov_matrix)
optimal_sharpe_ratio = sharpe_ratio(optimal_weights, log_returns, cov_matrix, risk_free_rate)

print(f"Expected Annual Return: {optimal_portfolio_return:.4f}")
print(f"Expected Portfolio Volatility: {optimal_portfolio_volatility:.4f}")
print(f"Expected Sharpe ratio: {optimal_sharpe_ratio:.4f}")

Optimal weights:
SPY: 0.5000
BND: 0.0000
GLD: 0.0000
QQQ: 0.5000
VTI: 0.0000

Expected Annual Return: 225.3163
Expected Portfolio Volatility: 16.4762
Expected Sharpe ratio: 13.6727
